In [78]:
import yfinance as yf
import pandas as pd
tickers=pd.read_csv('../DataCollection/Data/Index/nasdaq100.csv',index_col=0)
tickers = tickers.index.tolist()
data = yf.download(tickers, period="5y")['Close']

[*********************100%***********************]  101 of 101 completed


In [79]:
#drop na by columns
data = data.dropna(axis=1)
data.head()

Ticker,AAPL,ADBE,ADI,ADP,ADSK,AEP,AMAT,AMD,AMGN,AMZN,...,TSLA,TTD,TTWO,TXN,VRSK,VRTX,WBD,WDAY,XEL,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-03,79.009773,389.920013,111.721504,136.555298,226.389999,72.411667,55.280720,52.730000,189.135880,123.919998,...,58.863998,35.498001,129.300003,110.590309,165.501785,277.010010,22.270000,181.710007,57.260841,107.410004
2020-06-04,78.329338,385.799988,112.419647,135.015808,225.199997,70.255722,55.586723,52.630001,189.033188,123.029999,...,57.625332,33.723999,127.589996,111.781334,159.395767,272.820007,23.090000,175.300003,55.726898,103.050003
2020-06-05,80.560234,392.899994,114.449776,144.162704,233.710007,71.179688,57.556587,53.099998,193.132538,124.150002,...,59.043999,35.429001,128.910004,114.267677,161.014359,265.690002,24.780001,178.669998,55.924011,99.309998
2020-06-08,81.036537,397.779999,113.641388,145.080963,233.509995,73.177490,57.279278,52.970001,193.201050,126.203003,...,63.327999,35.831001,134.800003,115.502144,164.445343,265.010010,26.150000,181.229996,57.372231,102.739998
2020-06-09,83.595497,397.160004,113.347435,141.966003,234.149994,71.512657,57.174099,56.389999,192.473557,130.042999,...,62.711334,36.298000,134.410004,114.858849,164.115799,270.850006,23.879999,179.759995,56.875225,103.589996


In [ ]:
returns = data.pct_change().dropna()

volatility = returns.std()
avg_return = returns.mean()


from sklearn.cluster import KMeans
import pandas as pd

features = pd.DataFrame({
    'volatility': volatility,
    'avg_return': avg_return
})

kmeans = KMeans(n_clusters=3)
features['cluster'] = kmeans.fit_predict(features)


In [81]:
from sklearn.cluster import KMeans
import pandas as pd

features = pd.DataFrame({
    'volatility': volatility,
    'avg_return': avg_return
})

kmeans = KMeans(n_clusters=4)
features['cluster'] = kmeans.fit_predict(features)

In [82]:
features

,volatility,avg_return,cluster
Ticker,,,
AAPL,0.018941,0.000926,0
ADBE,0.022781,0.000290,0
ADI,0.020565,0.000734,0
ADP,0.014138,0.000793,2
ADSK,0.022242,0.000460,0
...,...,...,...
VRTX,0.018251,0.000548,0
WBD,0.034331,-0.000040,3
WDAY,0.024196,0.000538,0


In [83]:
#elbow method to find optimal number of clusters
import plotly.express as px
from sklearn.cluster import KMeans
inertia = []
n_samples = features.shape[0]
for i in range(1, n_samples + 1):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(features[['volatility', 'avg_return']])
    inertia.append(kmeans.inertia_)

fig = px.line(x=range(1, n_samples + 1), y=inertia, labels={'x': 'Number of Clusters', 'y': 'Inertia'})
fig.update_layout(title='Elbow Method for Optimal Clusters')
fig.show()

In [84]:
import plotly.express as px
#also show the ticker in the plot
features['Ticker'] = features.index
# Create a scatter plot with Plotly Express
fig = px.scatter(features, x='volatility', y='avg_return', color='cluster', text='Ticker',
                 title='Volatility vs Average Return Clustering',
                 labels={'volatility': 'Volatility', 'avg_return': 'Average Return'})
# Update layout for better readability
fig.update_traces(textposition='top center')
fig.update_layout(
    xaxis_title='Volatility',
    yaxis_title='Average Return',
    legend_title='Cluster'
)
fig.show()

In [85]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# สเกลข้อมูลก่อนทำ PCA
scaler = StandardScaler()
scaled_data = scaler.fit_transform(returns.T)  # T เพื่อให้หุ้นเป็น observations

# ทำ PCA
pca = PCA(n_components=2)  # ลดเหลือ 2 มิติ
pca_result = pca.fit_transform(scaled_data)

# ดูว่า Component ไหนอธิบายข้อมูลได้มากแค่ไหน
print("Explained variance ratio:", pca.explained_variance_ratio_)

Explained variance ratio: [0.14680614 0.07225861]


In [87]:
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'], index=returns.columns)

# สร้าง scatter plotly
fig = px.scatter(pca_df, x='PC1', y='PC2', text=pca_df.index,
                 title='PCA of Stock Returns',
                 labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'})
fig.update_traces(textposition='top center')
fig.update_layout(
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2'
)
fig.show()
